# Medical Q&A Chatbot Development Notebook

In [1]:
!which python

/c/Users/dubey/OneDrive/Desktop/Coding/Projects/Medical_Q&A_Chatbot/.conda/python


In [2]:
!pip install pandas scikit-learn nltk spacy scispacy
!python -m spacy download en_core_web_sm
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.3.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached cloudpathlib-0.21.0-py3-none-any.whl.metadata (14 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.6 MB 1.4 MB/s eta 0:00:08
   --- ------------------------------------ 1.0/11.6 MB 1.8 MB/s eta 0:00:06
   ----- ---------------------------------- 1.6/11.6 MB 2.0 MB/s eta 0:00:05
   ------- --------------

  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [517 lines of output]
        Using cached cymem-2.0.11-cp311-cp311-win_amd64.whl.metadata (8.8 kB)
        Using cached preshed-3.0.9-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
        Using cached murmurhash-1.0.12-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached blis-0.7.11-cp311-cp311-win_amd64.whl.metadata (7.6 kB)
        Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl.metadata (60 kB)
        Using cached srsly-2.5.1-cp311-cp311-win_amd64.whl.metadata (20 kB)
    

In [3]:
# 3. Import Libraries
import os
import xml.etree.ElementTree as ET
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import os
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm
import os
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm

In [4]:
!pip install lxml tqdm

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.8 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/3.8 MB 3.1 MB/s eta 0:00:01
   ------------------- -------------------- 1.8/3.8 MB 3.1 MB/s eta 0:00:01
   ------------------------ --------------- 2.4/3.8 MB 3.2 MB/s eta 0:00:01
   ----------------------------------- ---- 3.4/3.8 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 3.4 MB/s eta 0:00:00


In [5]:
# 3. Data Loading with proper XML structure handling
data_dir = "MedQuAD"
qa_pairs = []

# Walk through all directories and files with progress bar
for root, _, files in tqdm(os.walk(data_dir), desc="Processing files"):
    for filename in files:
        if filename.endswith('.xml'):
            file_path = os.path.join(root, filename)
            try:
                # Parse XML file
                tree = ET.parse(file_path)
                root_elem = tree.getroot()
                
                # Find all QA pairs in the XML structure
                qa_section = root_elem.find('QAPairs')
                if qa_section is not None:
                    for qa in qa_section.findall('QAPair'):
                        question = qa.find('Question')
                        answer = qa.find('Answer')
                        
                        if question is not None and answer is not None:
                            q_text = question.text.strip() if question.text else ""
                            a_text = answer.text.strip() if answer.text else ""
                            
                            if q_text and a_text:  # Only add if both exist
                                qa_pairs.append({
                                    'question': q_text,
                                    'answer': a_text
                                })
            except Exception as e:
                print(f"\nError in {file_path}: {str(e)}")
                continue

# Create DataFrame and verify
df = pd.DataFrame(qa_pairs)
print(f"\nLoaded {len(df)} QA pairs")

if not df.empty:
    print("\nFirst QA pair:")
    print(f"Question: {df.iloc[0]['question']}")
    print(f"Answer: {df.iloc[0]['answer'][:100]}...")  # Show first 100 chars
else:
    print("\nNo QA pairs loaded. Check XML structure and file paths.")  

Processing files: 29it [00:06,  4.18it/s]


Loaded 16407 QA pairs

First QA pair:
Question: What is (are) Adult Acute Lymphoblastic Leukemia ?
Answer: Key Points
                    - Adult acute lymphoblastic leukemia (ALL) is a type of cancer in whi...


In [6]:
# 4. Text Preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    # Handle missing values
    if not isinstance(text, str):
        return ''
    
    # Lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Tokenization and lemmatization
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to questions
df['processed_question'] = df['question'].apply(preprocess)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dubey\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dubey\AppData\Roaming\nltk_data...


In [7]:
# 5. TF-IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize and fit TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df['processed_question'])

In [8]:
# 6. Answer Retrieval Function
def get_most_similar_answer(query):
    # Preprocess the query
    processed_query = preprocess(query)
    # Vectorize the query
    query_vec = vectorizer.transform([processed_query])
    # Calculate cosine similarities
    similarities = cosine_similarity(query_vec, X_tfidf)
    # Get index of most similar question
    most_similar_idx = similarities.argmax()
    # Return corresponding answer
    return df.iloc[most_similar_idx]['answer']


In [9]:
# 7. Test the System
test_question = "What are the symptoms of diabetes?"
print(f"\nTest Question: {test_question}")
print("Answer:", get_most_similar_answer(test_question))



Test Question: What are the symptoms of diabetes?
Answer: Diabetes is often called a "silent" disease because it can cause serious complications even before you have symptoms. Symptoms can also be so mild that you dont notice them. An estimated 8 million people in the United States have type 2 diabetes and dont know it, according to 2012 estimates by the Centers for Disease Control and Prevention (CDC). Common Signs Some common symptoms of diabetes are: - being very thirsty  - frequent urination  - feeling very hungry or tired  - losing weight without trying  - having sores that heal slowly  - having dry, itchy skin  - loss of feeling or tingling in the feet  - having blurry eyesight. being very thirsty frequent urination feeling very hungry or tired losing weight without trying having sores that heal slowly having dry, itchy skin loss of feeling or tingling in the feet having blurry eyesight. Signs of type 1 diabetes usually develop over a short period of time. The signs for type 2 d

In [10]:
# 8. Enhanced Medical Entity Recognition
import spacy
nlp = spacy.load("en_core_web_sm")

# Medical entity labels to look for (based on common medical terminology)
MEDICAL_LABELS = {
    'SYMPTOMS': ['pain', 'fever', 'headache', 'nausea', 'rash'],  # Add more symptoms
    'DISEASES': ['diabetes', 'cancer', 'leukemia', 'hypertension'],
    'TREATMENTS': ['chemotherapy', 'radiation', 'surgery', 'antibiotics']
}

def extract_medical_entities(text):
    doc = nlp(text)
    entities = []
    
    # Check for medical terms using both NER and keyword matching
    for ent in doc.ents:
        if ent.label_ in ['CHEMICAL', 'DISEASE']:  # spaCy's limited medical labels
            entities.append((ent.text, ent.label_))
    
    # Additional keyword-based matching
    for token in doc:
        text_lower = token.text.lower()
        for label, keywords in MEDICAL_LABELS.items():
            if text_lower in keywords:
                entities.append((token.text, label))
    
    # Remove duplicates
    return list(set(entities))

In [11]:
# Test Enhanced Entity Recognition
test_text = "I have headache and fever for three days"
print("\nEnhanced Entity Recognition Test:")
print(f"Text: {test_text}")
print("Entities:", extract_medical_entities(test_text))


Enhanced Entity Recognition Test:
Text: I have headache and fever for three days
Entities: [('headache', 'SYMPTOMS'), ('fever', 'SYMPTOMS')]


In [13]:
!pip install streamlit

   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.8 MB 2.1 MB/s eta 0:00:05
   ---- ----------------------------------- 1.0/9.8 MB 2.8 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/9.8 MB 3.3 MB/s eta 0:00:03
   ----------- ---------------------------- 2.9/9.8 MB 3.4 MB/s eta 0:00:03
   ---------------- ----------------------- 3.9/9.8 MB 3.8 MB/s eta 0:00:02
   -------------------- ------------------- 5.0/9.8 MB 3.9 MB/s eta 0:00:02
   ------------------------ --------------- 6.0/9.8 MB 4.2 MB/s eta 0:00:01
   ----------------------------- ---------- 7.3/9.8 MB 4.4 MB/s eta 0:00:01
   ----------------------------------- ---- 8.7/9.8 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.8 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   ---------------------

In [19]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz (120.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached spacy-3.2.6.tar.gz (1.1 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: still running...
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached thinc-8.0.17-cp311-cp311-win_amd64.whl
  Using cached wasabi-0.10.1-py3-none-any.whl.metadata (28 kB)
  Using cached pathy-0.11.0-py3-none-any.whl.metadata (16 kB)
  Using cached smart_open-6.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached pydantic-1.8.2-py3-none-any.whl.meta

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
altair 5.5.0 requires typing-extensions>=4.10.0; python_version < "3.14", but you have typing-extensions 4.5.0 which is incompatible.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.2.6 which is incompatible.
ipython 8.30.0 requires typing-extensions>=4.6; python_version < "3.12", but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.33.1 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
scispacy 0.5.5 requires spacy<3.8.0,>=3.7.0, but you have spacy 3.2.6 which is incompatible.
typing-inspection 0.4.0 requires typing-extensions>=4.12.0, but you have typing-extensions 4.5.0 which is incompatible.


In [28]:
pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz (120.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached spacy-3.2.6-cp311-cp311-win_amd64.whl
  Using cached thinc-8.0.17-cp311-cp311-win_amd64.whl
  Using cached wasabi-0.10.1-py3-none-any.whl.metadata (28 kB)
  Using cached typer-0.4.2-py3-none-any.whl.metadata (12 kB)
  Using cached pathy-0.11.0-py3-none-any.whl.metadata (16 kB)
  Using cached pydantic-1.8.2-py3-none-any.whl.metadata (103 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached pathlib_abc-0.1.1-py3-none-any.whl.metadata (18 kB)
Using cached pathy-0.11.0-py3-none-any.whl (47 kB)
Using cached pathlib_abc-0.1.1-py3-none-any.whl (23 kB)
Using cached pydantic-1.8.2-py3-none-any.whl (126 kB)
Using cached typer-0.4.2-py3-none-any.whl (27 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
Usi

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'c:\\Users\\dubey\\OneDrive\\Desktop\\Coding\\Projects\\Medical_Q&A_Chatbot\\.conda\\Scripts\\spacy.exe'
Consider using the `--user` option or check the permissions.



In [26]:
!pip install -r requirements.txt

  Using cached streamlit-1.44.1-py3-none-any.whl.metadata (8.9 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached pydantic-1.10.13-cp311-cp311-win_amd64.whl.metadata (150 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.12-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
  Using cached cymem-2.0.11-cp311-cp311-win_amd64.whl.metadata (8.8 kB)
  Using cached preshed-3.0.9-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
  Using cached thinc-8.2.5-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached srsly-2.5.1-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached smart_open-6.4.0-py3-none-any.whl.metadata (21 kB)
  Using cac

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\dubey\\OneDrive\\Desktop\\Coding\\Projects\\Medical_Q&A_Chatbot\\.conda\\Scripts\\streamlit.exe' -> 'C:\\Users\\dubey\\OneDrive\\Desktop\\Coding\\Projects\\Medical_Q&A_Chatbot\\.conda\\Scripts\\streamlit.exe.deleteme'
Consider using the `--user` option or check the permissions.



In [29]:
!streamlit run med_chatbot.py

^C
